# 13-2. 자원을 공유할 때 - shared_ptr와 weak_ptr

`unique_ptr` : 객체를 유일하게 소유하는 스마트 포인터.

대부분의 경우, 하나의 자원은 한 개의 스마트 포인터에 의해 소유되는 것이 바람직 하고, 나머지 접근은 (소유가 아닌) 그냥 일반 포인터로 처리하면 된다.

하지만, 때에 따라 여러 개의 스마트 포인터가 하나의 객체를 같이 소유해야 하는 경우가 발생. 그 때, 좀 더 스마트한 포인터가 있어서 특정 자원을 몇 개 객체에서 가리키는지를 추적한 다음에, 그 수(참조 개수; reference count)가 0이 되어야만 비로소 해제를 시켜주는 방식의 포인터가 필요할 것.

## shared_ptr

`shared_ptr`은 앞서 이야기한 방식을 정확히 수행하는 스마트 포인터이다.

In [ ]:
std::shared_ptr<A> p1(new A());
std::shared_ptr<A> p2(p1);  // p2 역시 생성된 객체 A를 가리킨다.

// 반면에 unique_ptr의 경우
std::unique_ptr<A> p1(new A());
std::unique_ptr<A> p2(p1);  // 컴파일 오류!

In [ ]:
#include <iostream>
#include <memory>
#include <vector>

class A {
    int *data;
    
public:
    A() {
        data = new int[100];
        std::cout << "자원을 획득함!" << std::endl;
    }
    
    ~A() {
        std::cout << "소멸자 호출!" << std::endl;
        delete[] data;
    }
};

int main() {
    std::vector<std::shared_ptr<A>> vec;
    
    vec.push_back(std::shared_ptr<A>(new A()));
    vec.push_back(std::shared_ptr<A>(vec[0]));
    vec.push_back(std::shared_ptr<A>(vec[1]));
    
    // 벡터의 첫번째 원소를 소멸시킨다.
    std::cout << "첫 번째 소멸!" << std::endl;
    vec.erase(vec.begin());
    
    // 그 다음 원소를 소멸시킨다.
    std::cout << "다음 원소 소멸!" << std::endl;
    vec.erase(vec.begin());
    
    // 마지막 원소 소멸
    std::cout << "마지막 원소 소멸!" << std::endl;
    vec.erase(vec.begin());
    
    std::cout << "프로그램 종료!" << std::endl;
}

\- 실행 결과
```
자원을 획득함!
첫 번째 소멸!
다음 원소 소멸!
마지막 원소 소멸!
소멸자 호출!  // 참조 개수가 3->2->1->0으로 줄어들어서 소멸자 호출한 것!
프로그램 종료!
```

`shared_ptr`의 참조 개수가 몇 개인지는 `use_count()`를 통해 알 수 있다.

In [ ]:
std::shared_ptr<A> p1(new A());
std::shared_ptr<A> p2(p1);  // p2 역시 생성된 객체 A를 가리킨다.

std::cout << p1.use_count();  // 2
std::cout << p2.use_count();  // 2

![제어 블록](figures/Screen%20Shot%202021-01-22%20at%2011.02.18%20PM.png)

## make_shared로 생성하자

`std::shared_ptr<A> p1(new A());`는 사실 바람직한 `shared_ptr`의 생성 방법은 아니다.

일단 `A`를 생성하기 위해 동적 할당이 한 번 일어나고, 그 다음 `shared_ptr`의 제어 블록 역시 동적으로 할당해야 하기 때문. 즉, 두 번의 동적 할당이 발생해야 한다.

동적 할당은 상당히 비싼 연산이므로 어차피 두 번 할 것을 알고 있다면, 아예 두 개 합친 크기로 한 번 할당하는 것이 훨씬 빠르다.

In [ ]:
std::shared_ptr<A> p1 = std::make_shared<A>();

`make_shared` 함수는 `A`의 생성자의 인자들을 받아서 이를 통해 객체 `A`와 `shared_ptr`의 제어 블록까지 한 번에 동적 할당한 후에 만들어진 `shared_ptr`을 리턴한다.

`A`의 생성자에 인자가 있다면 `make_shared`에 인자로 전달해주면 된다.(그리고 `make_shared`가 `A`의 생성자에 완벽한 전달을 해줄 것.)

## shared_ptr 생성 시 주의할 점

`shared_ptr`은 인자로 주소값이 전달된다면, 마치 자기가 해당 객체를 첫번째로 소유하는 `shared_ptr`인 것 마냥 행동한다.

In [ ]:
A *a = new A();
std::shared_ptr<A> pa1(a);
std::shared_ptr<A> pa2(a);

![실행 결과](figures/Screen%20Shot%202021-01-22%20at%2011.12.18%20PM.png)

그래서 이와 같은 상황을 방지하려면 `shared_ptr`을 주소값을 통해서 생성하는 것을 지양해야 한다.

그런데, 어쩔 수 없는 상황도 있다. 바로 객체 내부에서 자기 자신을 가리키는 `shared_ptr`을 만들 때를 생각하자.

In [ ]:
class A {
    // ...
    
    std::shared_ptr<A> get_shared_ptr() { return std::shared_ptr<A>(this); }
};

int main() {
    std::shared_ptr<A> pa1 = std::make_shared<A>();
    std::shared_ptr<A> pa2 = pa1->get_shared_ptr();
    
    std::cout << pa1.use_count() << std::endl;  // 1
    std::cout << pa2.use_count() << std::endl;  // 1
}
// 런타임 오류! 소멸자가 두 번 호출되므로.

이러한 문제는 `enable_shared_from_this`를 통해 깔끔하게 해결할 수 있다!

만약 `this`를 사용해서 `shared_ptr`을 만들고 싶은 클래스가 있다면 `enable_shared_from_this`를 상속받으면 된다.

In [ ]:
#include <iostream>
#include <memory>

class A : public std::enable_shared_from_this<A> {
    int *data;

public:
    A() {
        data = new int[100];
        std::cout << "자원을 획득함!" << std::endl;
    }
    
    ~A() {
        std::cout << "소멸자 호출!" << std::endl;
        delete[] data;
    }
    
    std::shared_ptr<A> get_shared_ptr() { return shared_from_this(); }
};

int main() {
    std::shared_ptr<A> pa1 = std::make_shared<A>();
    std::shared_ptr<A> pa2 = pa1->get_shared_ptr();
    
    std::cout << pa1->get_shared_ptr();
    
    std::cout << pa1.use_count() << std::endl;
    std::cout << pa2.use_count() << std::endl;
}

\- 실행 결과
```
자원을 획득함!
2
2
소멸자 호출!
```

`enable_shared_from_this` 클래스에는 `shared_from_this`라는 멤버 함수를 정의하고 있는데, 
이 함수는 이미 정의되어 있는 제어 블록을 사용해서 `shared_ptr`을 생성한다. 따라서 이전처럼 같은 객체에 두 개의 다른 제어 블록이 생성되는 일을 막을 수 있다.

한 가지 중요한 점은 `shared_from_this`가 잘 작동하기 위해서는 해당 객체의 `shared_ptr`가 반드시 먼저 정의되어 있어야 한다. 즉, `shared_from_this`는 있는 블록만 확인을 할 뿐, 없는 블록을 만들지는 않는다. 쉽게 말해, 아래 코드는 오류가 발생한다.

In [ ]:
A *a = new A();
std::shared_ptr<A> pa1 = a->get_shared_ptr();  // 오류

## 서로 참조하는 shared_ptr

`shared_ptr`는 참조 개수가 0이 되면 가리키는 객체를 메모리에서 해제시키는데, 객체들을 더 이상 사용하지 않는데도 불구하고 참조 개수가 절대로 0이 될 수 없는 상황이 있다.

![참조 개수가 절대로 0이 될 수 없는 상황](figures/Screen%20Shot%202021-01-23%20at%2012.10.22%20AM.png)

In [ ]:
#include <iostream>
#include <memory>

class A {
    int *data;
    std::shared_ptr<A> other;
    
public:
    A() {
        data = new int[100];
        std::cout << "자원을 획득함!" << std::endl;
    }
    ~A() {
        std::cout << "소멸자 호출!" << std::endl;
        delete[] data;
    }
    
    void set_other(std::shared_ptr<A> o) { other = o; }
};

int main() {
    std::shared_ptr<A> pa = std::make_shared<A>();
    std::shared_ptr<A> pb = std::make_shared<A>();
    
    pa->set_other(pb);
    pb->set_other(pa);
}

\- 실행 결과
```
자원을 획득함!
자원을 획득함!
```

위와 같이 소멸자가 제대로 호출되지 않음을 알 수 있다. 이 문제는 `shared_ptr` 자체에 내재되어 있는 문제이므로 `shared_ptr`을 통해서는 해결할 수 없다. 이러한 순환 참조 문제를 해결하기 위해 나타난 것이 `weak_ptr`이다.

## weak_ptr